In [14]:
import pandas as pd
# python 환경 : pip install google-api-python-client
# anaconda환경 : conda install -c conda-forge google-api-python-client
from googleapiclient.discovery import build

# 유트브 APi : https://developers.google.com/youtube/v3/getting-started?hl=ko

# 예시 유트브 링크 : https://www.youtube.com/watch?v=tZolEtoQyuY
# 자신 API KEY 사용
text_file = open('./API_KEY.txt', 'r')
api_key = text_file.readline()
# 비디오 아이디 : tZolEtoQyuY
video_id = 'tZolEtoQyuY'

# 댓글을 담을 변수
comments = list()
# youtebe, v3 api 사용 developerKey는 발급받은 키 하루 10000건 조회가능
api_obj = build('youtube', 'v3', developerKey=api_key)

# response = api_obj.videoCategories().list(part='snippet,id', regionCode='kr').execute()

# https://www.youtube.com/watch?part='snippet,replies'&vieoid='asdasd'&maxResults=100이와 같이 요청을 함 snippet필수속성, replies댓글
# response = api_obj.videos().list(part='snippet,id', chart='mostPopular', regionCode='kr', maxResults=10, videoCategoryId='20').execute()



In [15]:
video_id_list = ['cX4EOv-_AGs', '8JbpWTljkdk', 'fk8vvmHIWAA']

In [7]:
# video_id_list = list()
# comments = list()
# for i in range(10):
#     video_id_list.append(response['items'][i]['id'])
    
# video_id_list

NameError: name 'response' is not defined

In [16]:
for video_id in video_id_list:
    response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, maxResults=100).execute()
    while response:
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments.append([comment['textOriginal'], comment['authorDisplayName'], comment['publishedAt'], comment['updatedAt'], comment['likeCount']])
            if item['snippet']['totalReplyCount'] > 0:
                for reply_item in item['replies']['comments']:
                    reply = reply_item['snippet']
                    comments.append([reply['textOriginal'], reply['authorDisplayName'], reply['publishedAt'], reply['updatedAt'], reply['likeCount']])
 
        if 'nextPageToken' in response:
            response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, pageToken=response['nextPageToken'], maxResults=10000).execute()
        else:
            break
    

print(len(comments))

16072


In [17]:
columns = ['comment', 'author', 'create_at', 'update_at', 'like_count']
comments_df = pd.DataFrame(comments, columns=columns)

In [30]:
comments_df['comment'].isna()

0        False
1        False
2        False
3        False
4        False
         ...  
16067    False
16068    False
16069    False
16070    False
16071    False
Name: comment, Length: 16072, dtype: bool

In [28]:
comments_df.describe()

,like_count
count,16072.000000
mean,20.855276
std,314.400980
min,0.000000
25%,1.000000
50%,2.000000
75%,10.000000
max,34622.000000


In [27]:
comments_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16072 entries, 0 to 16071
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   comment     16072 non-null  object
 1   author      16072 non-null  object
 2   create_at   16072 non-null  object
 3   update_at   16072 non-null  object
 4   like_count  16072 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 627.9+ KB
